## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-21-21-50-25 +0000,nypost,"President Trump, Elon Musk reunite at Charlie ...",https://nypost.com/2025/09/21/us-news/presiden...
1,2025-09-21-21-47-24 +0000,wapo,Trump administration ‘wouldn’t be here’ withou...,https://www.washingtonpost.com
2,2025-09-21-21-47-24 +0000,wapo,Live updates: Republicans vow at Charlie Kirk’...,https://www.washingtonpost.com/politics/2025/0...
3,2025-09-21-21-44-56 +0000,nyt,Here is the latest.,https://www.nytimes.com/live/2025/09/21/us/cha...
4,2025-09-21-21-39-56 +0000,nyt,The Conservative Figures That Attended the Cha...,https://www.nytimes.com/2025/09/21/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2316/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,trump,35
6,kirk,14
5,charlie,13
136,palestinian,11
133,canada,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
54,2025-09-21-17-51-06 +0000,nypost,Charlie Kirk funeral hits capacity as 200K peo...,https://nypost.com/2025/09/21/us-news/charlie-...,81
0,2025-09-21-21-50-25 +0000,nypost,"President Trump, Elon Musk reunite at Charlie ...",https://nypost.com/2025/09/21/us-news/presiden...,80
2,2025-09-21-21-47-24 +0000,wapo,Live updates: Republicans vow at Charlie Kirk’...,https://www.washingtonpost.com/politics/2025/0...,76
31,2025-09-21-20-00-00 +0000,wsj,The Trump administration’s moves to upend the ...,https://www.wsj.com/politics/policy/trumps-100...,65
1,2025-09-21-21-47-24 +0000,wapo,Trump administration ‘wouldn’t be here’ withou...,https://www.washingtonpost.com,64


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
54,81,2025-09-21-17-51-06 +0000,nypost,Charlie Kirk funeral hits capacity as 200K peo...,https://nypost.com/2025/09/21/us-news/charlie-...
88,51,2025-09-21-13-36-12 +0000,nypost,"Canada, UK and Australia all recognize Palesti...",https://nypost.com/2025/09/21/world-news/canad...
58,32,2025-09-21-17-12-42 +0000,nypost,Man charged with shooting up Sacramento ABC st...,https://nypost.com/2025/09/21/us-news/fbi-arre...
72,32,2025-09-21-15-39-29 +0000,nypost,Sky Meadow Country Club suspect Hunter Nadeau’...,https://nypost.com/2025/09/21/us-news/sky-mead...
31,30,2025-09-21-20-00-00 +0000,wsj,The Trump administration’s moves to upend the ...,https://www.wsj.com/politics/policy/trumps-100...
28,28,2025-09-21-20-11-42 +0000,nypost,Zohran Mamdani gives Rudy Giuliani some credit...,https://nypost.com/2025/09/21/us-news/zohran-m...
43,23,2025-09-21-18-50-48 +0000,nypost,Trump warns US will defend Poland and Baltics ...,https://nypost.com/2025/09/21/us-news/trump-wa...
157,23,2025-09-21-01-22-42 +0000,nypost,Elon Musk resurfaces Harris tweet calling for ...,https://nypost.com/2025/09/20/us-news/elon-mus...
16,22,2025-09-21-20-55-16 +0000,nypost,Nassau County DA demands local Dems distance t...,https://nypost.com/2025/09/21/us-news/nassau-c...
163,22,2025-09-21-00-28-45 +0000,nypost,Amazon recalls 500K consumer products: here ar...,https://nypost.com/2025/09/20/business/amazon-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
